In [101]:
import numpy as np
import pandas as pd
from math import isnan
from sklearn import preprocessing
from sklearn.impute import KNNImputer, SimpleImputer
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler

from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer

import copy

Loader

In [2]:
import json
def importFile(file_path):

    #TODO : ability to set other parameters when loading data through constructor
    
    if file_path.endswith('.csv') or file_path.endswith('.txt'):
        with open(file_path, 'r') as f:
            first_line = f.readline()
        if '\t' in first_line:
            return pd.read_csv(file_path, sep='\t')
        elif ';' in first_line:
            return pd.read_csv(file_path, sep=',')
        elif ',' in first_line:
            return pd.read_csv(file_path, sep=',')
        else:
            return pd.read_fwf(file_path)

    elif file_path.endswith('.json'):
        with open(file_path) as json_file:
            data = json.load(json_file)
        return pd.DataFrame(data)

    elif file_path.endswith('.xlsx'):
        excel_file = pd.ExcelFile(file_path)
        sheets = excel_file.sheet_names
        number_of_sheets = len(sheets)
        print(f'found {number_of_sheets} sheets ')
        if number_of_sheets == 1:
            return pd.read_excel(file_path, sheet_name=sheets[0])
        else:
            df = []
            for i in range(number_of_sheets):
                df.append(pd.read_excel(file_path, sheet_name=sheets[i]))
            return df 

    # TODO connecting to database and load sql tables in dataframe
    elif file_path.endswith('.sql'):
        return pd.read_sql(file_path)

    else:
        # TODO return a message such that it doesn't abruptly stop the program (error handling)
        return None


In [3]:
df = importFile('./datasets/MarketSegmentation_Train.csv')
df.head(5)

,ID,Gender,Ever_Married,Age,Graduated,Profession,Work_Experience,Spending_Score,Family_Size,Var_1,Segmentation
0,462809,Male,No,22,No,Healthcare,1.0,Low,4.0,Cat_4,D
1,462643,Female,Yes,38,Yes,Engineer,NaN,Average,3.0,Cat_4,A
2,466315,Female,Yes,67,Yes,Engineer,1.0,Low,1.0,Cat_6,B
3,461735,Male,Yes,67,Yes,Lawyer,0.0,High,2.0,Cat_6,B
4,462669,Female,Yes,40,Yes,Entertainment,NaN,High,6.0,Cat_6,A


In [6]:
# Handling missing values

df.isna().sum()

ID                   0
Gender               0
Ever_Married       140
Age                  0
Graduated           78
Profession         124
Work_Experience    829
Spending_Score       0
Family_Size        335
Var_1               76
Segmentation         0
dtype: int64

In [7]:
# drop row with null values
df = df.dropna(how='all') 
# reset the index of a pandas dataframe to default sequential values starting from 0.
# drop = True means that the old index is dropped and not included as a new column in the dataframe.
df = df.reset_index(drop=True)

In [8]:
df.columns[df.isnull().any()]

Index(['Ever_Married', 'Graduated', 'Profession', 'Work_Experience',
       'Family_Size', 'Var_1'],
      dtype='object')

In [80]:
num_cols = df.select_dtypes(include=np.number).columns

In [125]:
(df.dropna()['Work_Experience'] % 1 == 0).all()

True

In [127]:

def int_checker(df,df1,cols):
    for col in cols:
        # remove na values in original and check if all values have 0 decimal.
        if (df.dropna()[col] % 1 == 0).all():
    # if all values are integers, convert the column to int64 data type
            df1[col] = df1[col].round()
            df1[col] = df1[col].astype('int64')

In [126]:
def missing_values(df1, type="both", method_num="KNNImputer", method_cat="most_frequent"):
    df = copy.deepcopy(df1)

    num_cols = df.select_dtypes(include=np.number).columns
    cat_cols = df.select_dtypes(include=object).columns
    missing_num = missing_cat = 0

    if type == "both":
        mask = df.isnull().any()
        missing_num = missing_cat = 1
    elif type == "categorical":
        missing_cat = 1
        mask = df.dtypes == object
    elif type == "numerical":
        missing_num = 1
        mask = df.dtypes != object
    else:
        raise ValueError("Invalid value for type_of_feature")
        
# TODO multivariate imputation
    if missing_num:
        if method_num == 'KNNImputer':
            # TODO n neighbours
            imputer = KNNImputer()
            df[num_cols] = pd.DataFrame(imputer.fit_transform(
                copy.deepcopy(df[num_cols])), columns=num_cols)
            int_checker(df1, df, num_cols)

        elif method_num in ['mean', 'median', 'most_frequent']:
            imputer = SimpleImputer(strategy=method_num)
            df[num_cols] = pd.DataFrame(imputer.fit_transform(
                copy.deepcopy(df[num_cols])), columns=num_cols)
            int_checker(df1, df, num_cols)

        elif method_num == "IterativeImputer":
            imputer = IterativeImputer(max_iter=10, random_state=0)
            df[num_cols] = pd.DataFrame(imputer.fit_transform(
                copy.deepcopy(df[num_cols])), columns=num_cols)
            int_checker(df1, df, num_cols)
        else:
            pass

    if missing_cat:
        if method_cat == 'KNNImputer':
            pass
        elif method_cat in ['most_frequent']:
            imputer = SimpleImputer(strategy=method_cat)
            df[cat_cols] = pd.DataFrame(imputer.fit_transform(
                copy.deepcopy(df[cat_cols])), columns=cat_cols)
        else:
            pass
    return df


In [128]:
df1 = missing_values(df,"both",)
df1.isna().sum().sum()

0

In [131]:
df1

,ID,Gender,Ever_Married,Age,Graduated,Profession,Work_Experience,Spending_Score,Family_Size,Var_1,Segmentation
0,462809,Male,No,22,No,Healthcare,1,Low,4,Cat_4,D
1,462643,Female,Yes,38,Yes,Engineer,3,Average,3,Cat_4,A
2,466315,Female,Yes,67,Yes,Engineer,1,Low,1,Cat_6,B
3,461735,Male,Yes,67,Yes,Lawyer,0,High,2,Cat_6,B
4,462669,Female,Yes,40,Yes,Entertainment,3,High,6,Cat_6,A
...,...,...,...,...,...,...,...,...,...,...,...
8063,464018,Male,No,22,No,Artist,0,Low,7,Cat_1,D
8064,464685,Male,No,35,No,Executive,3,Low,4,Cat_4,D
8065,465406,Female,No,33,Yes,Healthcare,1,Low,1,Cat_6,D
8066,467299,Female,No,27,Yes,Healthcare,1,Low,4,Cat_6,B


In [8]:
df.select_dtypes(include=np.number).columns

Index(['ID', 'Age', 'Work_Experience', 'Family_Size'], dtype='object')

In [9]:
import copy
df1 = copy.deepcopy(df)

In [41]:
def lin_regression_impute(df, model):
        # function for predicting missing values with linear regression

        #  gets a list of all the column names that contain numerical data in the DataFrame.
        cols_num = df.select_dtypes(include=np.number).columns

        #  empty dictionary mapping to store the mapping between categorical feature values and their numerical representations.
        mapping = dict()

        for feature in df.columns:
            if feature not in cols_num:

                # create label mapping for categorical feature values
                # this line creates a dictionary mappings that maps each
                #  unique value in the column to a unique integer representation, using the enumerate function.
                    # mapping = {}
                    # for i, k in enumerate(df1['Gender']):
                    #   mapping[k] = i
                mappings = {k: i for i, k in enumerate(df[feature])} 
                mapping[feature] = mappings

                # maps the dictionary to that feature column
                df[feature] = df[feature].map(mapping[feature])

                # this is also converting nan values to numbers which is handleled by knn assuming

        for feature in cols_num: 
                try:

                     # dataframe of rows that have na values for that feature. All the rows containig na values for other features are dropped.
                    test_df = df[df[feature].isnull()==True].dropna(subset=[x for x in df.columns if x != feature])

                    # dataframe of rows that do not have na values for that feature. All the rows containig na values for other features are dropped.
                    train_df = df[df[feature].isnull()==False].dropna(subset=[x for x in df.columns if x != feature])
                    if len(test_df.index) != 0:

                        # assuming its making a sequence of things to do. Here, its fitting to the dataframe.
                        # StandardScaler will standardize the independent variables (mean = 0, SD = 1) but does not change skewness of the data i.e make it gaussian
                        # Its often not necessary to make independent variables into gaussian distribution but may benefit in some cases,
                        pipe = make_pipeline(StandardScaler(), model)

                        # applying log tranformation to target variable to make it gaussian distribution
                        y = np.log(train_df[feature]) # log-transform the data
                        X_train = train_df.drop(feature, axis=1)
                        test_df.drop(feature, axis=1, inplace=True)
                        
                        try:
                            model = pipe.fit(X_train, y)  #might return error because logarithm of zero or negative values is undefined
                        except:
                            y = train_df[feature] # use non-log-transformed data
                            model = pipe.fit(X_train, y)

                         #  checks if the target variable used for fitting the model is the non-log-transformed one or not. If the target variable is non-log-transformed, then the code will proceed to the next step.
                        if (y == train_df[feature]).all(): 
                            pred = model.predict(test_df)
                        else:
                            # If the target variable used for fitting the model is the log-transformed one, then the predictions are exponentiated to get back the original target variable values.
                            pred = np.exp(model.predict(test_df)) # predict values

                        test_df[feature]= pred

#  checks if all the values in the target variable, after replacing the missing values with -9999, are evenly divisible by 1. 
# If this condition is true, it means that all the values are integers, and if it's false, it means that at least one of the values is a floating-point number. 
# The .all() method at the end checks if the condition is true for all the values in the target variable.
# -9999 is a value that is highly unlikely to appear in the original data
                        if (df[feature].fillna(-9999) % 1  == 0).all(): # checks if the original target variable was an integer
                            # round back to INTs, if original data were INTs
                            test_df[feature] = test_df[feature].round()
                            test_df[feature] = test_df[feature].astype('Int64')
                            df[feature].update(test_df[feature])                          
                        else:
                            df[feature].update(test_df[feature])  
                        print(f'LINREG imputation of {len(pred)} value(s) succeeded for feature "{feature}"')
                except:
                    print(f'LINREG imputation failed for feature "{feature}"', )
        for feature in df.columns: 
            try:   
                # map categorical feature values back to original
# mappings_i = {}
# for k, v in  mapping['Profession'].items():
#     mappings_i[v] = k
# mappings_i
# {8066: 'Healthcare',
#  8048: 'Engineer',
#  8057: 'Lawyer',..}

                mappings_inv = {v: k for k, v in mapping[feature].items()}
                df[feature] = df[feature].map(mappings_inv)
            except:
                pass
        return df

def impute( df, imputer, type):
        # function for imputing missing values in the data
        cols_num = df.select_dtypes(include=np.number).columns 

        if type == 'num':
            # numerical features
            for feature in df.columns: 
                if feature in cols_num:
                    if df[feature].isna().sum().sum() != 0:
                        try:
                            df_imputed = pd.DataFrame(imputer.fit_transform(np.array(df[feature]).reshape(-1, 1)))
                            counter = df[feature].isna().sum().sum() - df_imputed.isna().sum().sum()

                            if (df[feature].fillna(-9999) % 1  == 0).all():
                                df[feature] = df_imputed
                                # round back to INTs, if original data were INTs
                                df[feature] = df[feature].round()
                                df[feature] = df[feature].astype('Int64')                                        
                            else:
                                df[feature] = df_imputed
                            if counter != 0:
                                print(f' imputation of {counter} value(s) succeeded for feature "{feature}"' )
                        except:
                            print(f'imputation failed for feature "{feature}"')
        else:
            # categorical features
            for feature in df.columns:
                if feature not in cols_num:
                    if df[feature].isna().sum()!= 0:
                        try:
                            mapping = dict()
                            mappings = {k: i for i, k in enumerate(df[feature].dropna().unique(), 0)}
                            mapping[feature] = mappings
                            df[feature] = df[feature].map(mapping[feature])

                            df_imputed = pd.DataFrame(imputer.fit_transform(np.array(df[feature]).reshape(-1, 1)), columns=[feature])    
                            counter = sum(1 for i, j in zip(list(df_imputed[feature]), list(df[feature])) if i != j)

                            # round to integers before mapping back to original values
                            df[feature] = df_imputed
                            df[feature] = df[feature].round()
                            df[feature] = df[feature].astype('Int64')  

                            # map values back to original
                            mappings_inv = {v: k for k, v in mapping[feature].items()}
                            df[feature] = df[feature].map(mappings_inv)
                            if counter != 0:
                               print(f'imputation of {counter} value(s) succeeded for feature "{feature}"' )
                        except:
                            print(f'imputation failed for feature "{feature}"' )
        return df

In [62]:
lr = LinearRegression()
df2 = lin_regression_impute(copy.deepcopy(df), lr)


LINREG imputation of 764 value(s) succeeded for feature "Work_Experience"
LINREG imputation of 270 value(s) succeeded for feature "Family_Size"


c:\Users\deepdesk\anaconda3\lib\site-packages\pandas\core\arraylike.py:397: RuntimeWarning: divide by zero encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)


In [63]:
df2.isna().sum()

ID                   0
Gender               0
Ever_Married       140
Age                  0
Graduated           78
Profession         124
Work_Experience     65
Spending_Score       0
Family_Size         65
Var_1               76
Segmentation         0
dtype: int64

In [65]:
df2[df2['Work_Experience'].isnull()==True]

,ID,Gender,Ever_Married,Age,Graduated,Profession,Work_Experience,Spending_Score,Family_Size,Var_1,Segmentation
186,459576,Female,Yes,85,No,Lawyer,NaN,Low,NaN,NaN,A
352,467426,Female,Yes,86,No,Lawyer,NaN,Low,NaN,Cat_6,B
652,465638,Female,Yes,51,Yes,Artist,NaN,Low,NaN,Cat_3,C
766,461233,Male,Yes,37,Yes,Entertainment,NaN,Low,NaN,Cat_3,D
943,459453,Male,Yes,88,Yes,Lawyer,NaN,Low,NaN,Cat_6,B
...,...,...,...,...,...,...,...,...,...,...,...
7440,459620,Female,Yes,29,No,Marketing,NaN,Low,NaN,NaN,D
7493,459783,Female,Yes,18,No,Artist,NaN,High,NaN,Cat_1,B
7530,459057,Male,Yes,68,No,Lawyer,NaN,High,NaN,Cat_6,D
7607,462779,Female,Yes,61,No,Marketing,NaN,Average,NaN,Cat_6,D


In [66]:
df2[df2['Family_Size'].isnull()==True]

,ID,Gender,Ever_Married,Age,Graduated,Profession,Work_Experience,Spending_Score,Family_Size,Var_1,Segmentation
186,459576,Female,Yes,85,No,Lawyer,NaN,Low,NaN,NaN,A
352,467426,Female,Yes,86,No,Lawyer,NaN,Low,NaN,Cat_6,B
652,465638,Female,Yes,51,Yes,Artist,NaN,Low,NaN,Cat_3,C
766,461233,Male,Yes,37,Yes,Entertainment,NaN,Low,NaN,Cat_3,D
943,459453,Male,Yes,88,Yes,Lawyer,NaN,Low,NaN,Cat_6,B
...,...,...,...,...,...,...,...,...,...,...,...
7440,459620,Female,Yes,29,No,Marketing,NaN,Low,NaN,NaN,D
7493,459783,Female,Yes,18,No,Artist,NaN,High,NaN,Cat_1,B
7530,459057,Male,Yes,68,No,Lawyer,NaN,High,NaN,Cat_6,D
7607,462779,Female,Yes,61,No,Marketing,NaN,Average,NaN,Cat_6,D


In [45]:

imputer = KNNImputer(n_neighbors=3)
df3 = impute( copy.deepcopy(df2), imputer, type='num')

 imputation of 65 value(s) succeeded for feature "Work_Experience"
 imputation of 65 value(s) succeeded for feature "Family_Size"


In [46]:
df3.isna().sum()

ID                   0
Gender               0
Ever_Married       140
Age                  0
Graduated           78
Profession         124
Work_Experience      0
Spending_Score       0
Family_Size          0
Var_1               76
Segmentation         0
dtype: int64

In [10]:
cols_num = df.select_dtypes(include=np.number).columns
mapping = dict()
for feature in df.columns:
    if feature not in cols_num:
        # create label mapping for categorical feature values
        mappings = {k: i for i, k in enumerate(df1[feature])}
        mapping[feature] = mappings
        df1[feature] = df1[feature].map(mapping[feature])

In [11]:
mapping

{'Gender': {'Male': 8067, 'Female': 8066},
 'Ever_Married': {'No': 8066, 'Yes': 8067, nan: 8044},
 'Graduated': {'No': 8064, 'Yes': 8067, nan: 7995},
 'Profession': {'Healthcare': 8066,
  'Engineer': 8048,
  'Lawyer': 8057,
  'Entertainment': 8058,
  'Artist': 8062,
  'Executive': 8067,
  'Doctor': 8047,
  'Homemaker': 8056,
  'Marketing': 8026,
  nan: 8063},
 'Spending_Score': {'Low': 8066, 'Average': 8067, 'High': 8062},
 'Var_1': {'Cat_4': 8067,
  'Cat_6': 8066,
  'Cat_7': 8020,
  'Cat_3': 8058,
  'Cat_1': 8063,
  'Cat_2': 8053,
  nan: 7991,
  'Cat_5': 7967},
 'Segmentation': {'D': 8065, 'A': 8060, 'B': 8067, 'C': 8061}}

In [12]:
df1

,ID,Gender,Ever_Married,Age,Graduated,Profession,Work_Experience,Spending_Score,Family_Size,Var_1,Segmentation
0,462809,8067,8066,22,8064,8066,1.0,8066,4.0,8067,8065
1,462643,8066,8067,38,8067,8048,NaN,8067,3.0,8067,8060
2,466315,8066,8067,67,8067,8048,1.0,8066,1.0,8066,8067
3,461735,8067,8067,67,8067,8057,0.0,8062,2.0,8066,8067
4,462669,8066,8067,40,8067,8058,NaN,8062,6.0,8066,8060
...,...,...,...,...,...,...,...,...,...,...,...
8063,464018,8067,8066,22,8064,8063,0.0,8066,7.0,8063,8065
8064,464685,8067,8066,35,8064,8067,3.0,8066,4.0,8067,8065
8065,465406,8066,8066,33,8067,8066,1.0,8066,1.0,8066,8065
8066,467299,8066,8066,27,8067,8066,1.0,8066,4.0,8066,8067


In [19]:
mappings_i = {}
for k, v in  mapping['Profession'].items():
    mappings_i[v] = k
mappings_i

{8066: 'Healthcare',
 8048: 'Engineer',
 8057: 'Lawyer',
 8058: 'Entertainment',
 8062: 'Artist',
 8067: 'Executive',
 8047: 'Doctor',
 8056: 'Homemaker',
 8026: 'Marketing',
 8063: nan}

In [14]:
for feature in df1.columns: 
    try:
        # map categorical feature values back to original
        mappings_inv = {v: k for k, v in mapping[feature].items()}
        df1[feature] = df1[feature].map(mappings_inv)
    except:
        pass

In [15]:
df1

,ID,Gender,Ever_Married,Age,Graduated,Profession,Work_Experience,Spending_Score,Family_Size,Var_1,Segmentation
0,462809,Male,No,22,No,Healthcare,1.0,Low,4.0,Cat_4,D
1,462643,Female,Yes,38,Yes,Engineer,NaN,Average,3.0,Cat_4,A
2,466315,Female,Yes,67,Yes,Engineer,1.0,Low,1.0,Cat_6,B
3,461735,Male,Yes,67,Yes,Lawyer,0.0,High,2.0,Cat_6,B
4,462669,Female,Yes,40,Yes,Entertainment,NaN,High,6.0,Cat_6,A
...,...,...,...,...,...,...,...,...,...,...,...
8063,464018,Male,No,22,No,NaN,0.0,Low,7.0,Cat_1,D
8064,464685,Male,No,35,No,Executive,3.0,Low,4.0,Cat_4,D
8065,465406,Female,No,33,Yes,Healthcare,1.0,Low,1.0,Cat_6,D
8066,467299,Female,No,27,Yes,Healthcare,1.0,Low,4.0,Cat_6,B


In [53]:
mapping = {}
for i, k in enumerate(df1['Gender']):
    mapping[k] = i   #mappings['Male'] = 1....

{'Male': 8067, 'Female': 8066}


In [54]:
print({ k : i for i, k in enumerate(df['Gender']) })

{'Male': 8067, 'Female': 8066}


In [43]:
for feature in cols_num:    

    # dataframe of rows that have na values for that feature. All the rows containig na values for other features are dropped.
    test_df = df1[df1[feature].isnull()==True].dropna(subset=[x for x in df1.columns if x != feature])

    # dataframe of rows that do not have na values for that feature. All the rows containig na values for other features are dropped.
    train_df = df1[df1[feature].isnull()==False].dropna(subset=[x for x in df1.columns if x != feature])

In [44]:
len(test_df.index)

270

## encoding

In [29]:
#TODO ask if want to convert categorical to numerical
# categorical to numerical
df["Gender"] = df["Gender"].astype("category")
df["Gender_encoded"] = df["Gender"].cat.codes

## handling missing values

In [19]:
# mean imputation
df.mean()

C:\Users\deepdesk\AppData\Local\Temp\ipykernel_16060\3977765826.py:2: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  df.mean()


ID                 463479.214551
Age                    43.466906
Work_Experience         2.641663
Family_Size             2.850123
dtype: float64

In [20]:
df.median()

C:\Users\deepdesk\AppData\Local\Temp\ipykernel_16060\530051474.py:1: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  df.median()


ID                 463472.5
Age                    40.0
Work_Experience         1.0
Family_Size             3.0
dtype: float64

In [22]:
df.mode().iloc[0]

ID                 458982
Gender               Male
Ever_Married          Yes
Age                  35.0
Graduated             Yes
Profession         Artist
Work_Experience       1.0
Spending_Score        Low
Family_Size           2.0
Var_1               Cat_6
Segmentation            D
Name: 0, dtype: object

In [ ]:
mean_imputer = SimpleImputer(strategy='mean')
median_imputer = SimpleImputer(strategy='median')
most_frequent_imputer = SimpleImputer(strategy='most_frequent')

# Fit the imputer to the data and transform it
df_mean_imputed = pd.DataFrame(mean_imputer.fit_transform(df), columns=df.columns)
df_median_imputed = pd.DataFrame(median_imputer.fit_transform(df), columns=df.columns)
df_most_frequent_imputed = pd.DataFrame(most_frequent_imputer.fit_transform(df), columns=df.columns)

In [ ]:
knnimputer = KNNImputer(n_neighbors=5)
df_imputed = pd.DataFrame(knnimputer.fit_transform(df), columns=df.columns)

## Scaling

In [ ]:
# import pandas as pd
# from sklearn.preprocessing import MinMaxScaler

# # Load the data set
# df = pd.read_csv("data.csv")

# # Create the rescaler object
# scaler = MinMaxScaler()

# # Fit the rescaler to the data and transform it
# df_scaled = pd.DataFrame(scaler.fit_transform(df), columns=df.columns)
